In [8]:
from bs4 import BeautifulSoup
import string
import requests

from fake_useragent import UserAgent
ua = UserAgent()

In [9]:
def remove_punctuation(x):
    x = str(x)
    return x.translate(str.maketrans({a:None for a in string.punctuation}))

In [10]:
def get_soup(url, timeout=5):
    headers  = {'User-Agent':ua.random}
    try:
        response = requests.get(url,headers=headers)
    except:
        print("FAILED "+ url)
        return 0
    attempts = 0
    while(not response.ok):
            print((url+' failed with code: '+str(response.status_code)))
            if attempts > timeout:
                print(url+' failed with code: '+str(response.status_code))
                return BeautifulSoup('','lxml')
            response = requests.get(url)
            attempts += 1
    page = response.text
    soup = BeautifulSoup(page,'lxml')
    return soup

In [14]:
url = 'http://www.beeradvocate.com/beer/style/'
soup = get_soup(url)

In [32]:
beer_styles = {}
for style in soup.find('table').find_all('a'):
    beer_styles[style.get_text()] = style['href']

In [41]:
url = beer_styles.items()[0][1]
ba_url = 'http://www.beeradvocate.com'
soup = get_soup(ba_url+url)

In [86]:
beers = {}

ba_url = 'http://www.beeradvocate.com'
for style in beer_styles.items():
    style_url = style[1]
    soup = get_soup(ba_url+style_url)
    for beer_row in soup.find_all('tr')[3:-1]:
        beer_link = beer_row.find(class_ = 'hr_bottom_light').find('a')
        beers[beer_link.get_text()] = beer_link['href']

5071

In [ ]:
ba_url = 'http://www.beeradvocate.com'

In [90]:
def get_beer_reviews(soup):
    reviews = []
    for rating in soup.find_all(id='rating_fullview_content_2'):
        for span in rating.find_all('span'):
            span.extract()
        review = rating.get_text().strip().encode('utf-8')
        review = review.replace('rDev','')
        reviews.append(str(review))
    return reviews


In [91]:
beer_reviews = {}
ba_url = 'http://www.beeradvocate.com'

for beer in beers.items():
    soup = get_soup(ba_url+beer[1])
    print(beer[0])
    beer_reviews[beer[0]] = get_beer_reviews(soup)

Dreadnaught IPA
In Primus
Arrogant Bastard Ale
Red Dog
Vienna Red Lager
Smithwick's Imported Premium Irish Ale
Double Chocolate Stout
Moosbacher Schwarze Weisse
Misery
Simple Malt Double Porter
Roggen With The Devil
Hoppy Daze IPA
Sapporo Mugi To Hopppu (trans. Barley And Hop)
Love Is EvoL
Nikko Dark Beer
Worthington's White Shield
Brewers’ Cut Helles
Emelisse Black & Tan Port Charlotte BA
Serpent's Stout
J.W. Lees Harvest Ale (Sherry Cask)
Rare Vos (Amber Ale)
Dixie Blackened Voodoo Lager
Dorado Double IPA
Gouden Carolus Classic
Sunlight Cream Ale
Hose
Bombshell Blonde
Smuttynose Wheat Wine (Big Beer Series)
Allagash Four
Rieser Kristall-Weizen
Harvest Dance
Allagash Curieux (Bourbon Barrel-Aged Tripel)
Castelain Blond Biere De Garde
Whiplash White IPA
Ranger
Tempelier
Insanely Bad Elf
French Country Christmas Ale (Biere De Garde)
Founders Centennial IPA
Southampton Grand Cru
Atom Smasher
Abbey Dubbel
Dark Island Reserve
YuleSmith (Winter)
Joe
312 Urban Wheat
Stonewall Stout
Duck Duck

In [92]:
import pickle

In [94]:
pickle.dump(beer_reviews,open('beer_reviews.pkl','wb'))

In [95]:
pickle.dump(beers,open('beers.pkl','wb'))

In [96]:
pickle.dump(beer_styles,open('beer_styles.pkl','wb'))

In [ ]:
df.apply